### CNN-based 3D Skeleton Tennis Shot Recognition

Based on method from paper ['Skeleton-based Action Recognition with Convolutional Neural Networks'](https://arxiv.org/abs/1704.07595) by Li et al. (2017).

We use the 3D skeletal animation data from the ['THETIS'](http://thetis.image.ece.ntua.gr/) dataset for training. Each animation is represented as a `T x N` 3-channel image where `T` is the number of frames of the animation, `N` is the number of skeletal joints, and the 3 colour channels represent `x, y, z` coordinates of the joints.

We also use the motion data, represented in the same image format.

Load the skeleton and motion images training data.

In [4]:
import tensorflow as tf

skeleton_dir = "data/skeleton_images"
motion_dir = "data/motion_images"

img_height = 32
img_width = 32
batch_size = 32

skeleton_train_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

motion_train_ds = tf.keras.utils.image_dataset_from_directory(
  motion_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

skeleton_val_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

motion_val_ds = tf.keras.utils.image_dataset_from_directory(
  motion_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

skeleton_train_images = skeleton_train_ds.map(lambda img, label: img)
motion_train_images = motion_train_ds.map(lambda img, label: img)
train_images = tf.data.Dataset.zip((skeleton_train_images, motion_train_images))

train_labels = skeleton_train_ds.map(lambda img, label: label)
train_ds = tf.data.Dataset.zip((train_images, train_labels))

skeleton_val_images = skeleton_val_ds.map(lambda img, label: img)
motion_val_images = motion_val_ds.map(lambda img, label: img)
val_images = tf.data.Dataset.zip((skeleton_val_images, motion_val_images))

val_labels = skeleton_val_ds.map(lambda img, label: label)
val_ds = tf.data.Dataset.zip((val_images, val_labels))

Found 621 files belonging to 12 classes.
Using 497 files for training.
Found 621 files belonging to 12 classes.
Using 497 files for training.
Found 621 files belonging to 12 classes.
Using 124 files for validation.
Found 621 files belonging to 12 classes.
Using 124 files for validation.


Create the CNN model.

In [5]:
from tensorflow.keras import layers

skeleton_input = layers.Input(shape=(32,32,3))
skeleton_rescale = layers.Rescaling(1./127.5, offset=-1)(skeleton_input)
skeleton_conv1 = layers.Conv2D(32, (3,3), activation='relu')(skeleton_rescale)
skeleton_pool = layers.MaxPooling2D((2,2))(skeleton_conv1)
skeleton_conv2 = layers.Conv2D(64, (3,3), activation='relu')(skeleton_pool)

motion_input = layers.Input(shape=(32,32,3))
motion_rescale = layers.Rescaling(1./127.5, offset=-1)(motion_input)
motion_conv1 = layers.Conv2D(32, (3,3), activation='relu')(motion_rescale)
motion_pool = layers.MaxPooling2D((2,2))(motion_conv1)
motion_conv2 = layers.Conv2D(64, (3,3), activation='relu')(motion_pool)

concat = layers.Concatenate()([skeleton_conv2, motion_conv2])
flatten = layers.Flatten()(concat)
dense = layers.Dense(64, activation='relu')(flatten)
softmax = layers.Dense(12, activation='softmax')(dense)

model = tf.keras.Model(inputs=[skeleton_input, motion_input], outputs=[softmax])

Compile and train the model.

In [6]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

model.save('cnn_recognition_model.h5')

Epoch 1/100
16/16 [==============================] - 1s 37ms/step - loss: 2.4894 - accuracy: 0.0644 - val_loss: 2.4965 - val_accuracy: 0.0565
Epoch 2/100
16/16 [==============================] - 1s 32ms/step - loss: 2.4859 - accuracy: 0.0724 - val_loss: 2.5003 - val_accuracy: 0.0645
Epoch 3/100
16/16 [==============================] - 1s 32ms/step - loss: 2.4851 - accuracy: 0.0724 - val_loss: 2.5029 - val_accuracy: 0.0565
Epoch 4/100
16/16 [==============================] - 1s 32ms/step - loss: 2.4840 - accuracy: 0.0704 - val_loss: 2.5027 - val_accuracy: 0.0726
Epoch 5/100
16/16 [==============================] - 1s 33ms/step - loss: 2.4837 - accuracy: 0.0825 - val_loss: 2.5028 - val_accuracy: 0.0565
Epoch 6/100
16/16 [==============================] - 1s 32ms/step - loss: 2.4819 - accuracy: 0.0966 - val_loss: 2.4983 - val_accuracy: 0.0565
Epoch 7/100
16/16 [==============================] - 1s 32ms/step - loss: 2.4829 - accuracy: 0.0926 - val_loss: 2.4994 - val_accuracy: 0.0565
Epoch 